In [2]:
import pandas as pd

# Avito Duplicate Ads Detection

luoq08@gmail.com OR hzluoqiang@corp.netease.com

## data

![data](avito-dup.svg)

### size

* train pair: 2991396
* test pair: 1044196
* train info: 3344613
* train test: 1315205
* image: 11380670

## solution

[detail](solution.md)

### feature contribution

impact on final result(from old model)

In [4]:
pd.read_csv('auc.old.csv')

,model,features,test set,public leaderboard,gap
0,xgboost.26.weighted,all,0.956060,0.91957,0.036490
1,xgboost.27.weighted,-image,0.929910,0.86827,0.061640
2,xgboost.28.weighted,"-image, -corpus",0.922269,0.85488,0.067389
3,Xgboost.29.weighted,-corpus,0.952553,0.91441,0.038143
4,lr_text.1,title_description_dtm_0,0.793570,0.64333,0.150240
5,Xgboost.31.weighted,+description_sentence__binary__agg_cosine,0.956172,0.91963,0.036542


### feature importance

in final model(without)

### interesting finding

* gap between train and test
  * not iid
  * ordered by time
  * text feature 